## Data Augmentation

Check these out later: https://www.tutorialspoint.com/natural_language_toolkit/natural_language_toolkit_synonym_antonym_replacement.htm, https://www.kaggle.com/code/rohitsingh9990/data-augmentation-by-synonym-replacement

In [1]:
import pandas as pd
import numpy as np
import os


DATA_PATH = os.path.join("..", "data")

In [2]:
train_df = pd.read_csv(os.path.join(DATA_PATH, "train_essays.csv"))
train_df

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0


In [3]:
train_df[train_df.generated == 1]

,id,prompt_id,text,generated
704,82131f68,1,"This essay will analyze, discuss and prove one...",1
740,86fe4f18,1,I strongly believe that the Electoral College ...,1
1262,eafb8a56,0,"Limiting car use causes pollution, increases c...",1


In [4]:
def read_gpt(file: str) -> pd.Series:
    with open(os.path.join(DATA_PATH, file), encoding="utf8") as f:
        contents = f.read()
    responses = list(filter(lambda x: x.startswith(" ChatGPT"), 
                        contents.split("##")))
    clear_responses = [res.replace("ChatGPT", "").replace("~", "").strip() 
                            for res in responses]
    return pd.Series(clear_responses)


chat_gpt_environment = read_gpt("chatgpt_cars.md")
envir_df = pd.DataFrame({"text": chat_gpt_environment, 
                         "prompt_id": np.zeros_like(chat_gpt_environment),
                         "generated": np.ones_like(chat_gpt_environment)})

chat_gpt_electoral = read_gpt("chatgpt_electoral.md")
elect_df = pd.DataFrame({"text": chat_gpt_electoral, 
                         "prompt_id": np.ones_like(chat_gpt_electoral),
                         "generated": np.ones_like(chat_gpt_electoral)})

df = pd.concat([envir_df, 
                elect_df, 
                train_df.loc[:, ["text", "prompt_id", "generated"]]])
df

,text,prompt_id,generated
0,"In recent years, there has been a growing glob...",0,1
1,The advantages of limiting car usage are evide...,0,1
2,The global movement towards limiting car usage...,0,1
3,The advantages of limiting car usage are becom...,0,1
4,The advantages of limiting car usage are manif...,0,1
...,...,...,...
1373,There has been a fuss about the Elector Colleg...,1,0
1374,Limiting car usage has many advantages. Such a...,0,0
1375,There's a new trend that has been developing f...,0,0
1376,As we all know cars are a big part of our soci...,0,0


In [5]:
df.text.loc[df.text.apply(lambda x: x.startswith("Dear"))].iloc[3][-15]

'y'

In [6]:
import re


bracket_regex = r"\[.*?\]"

df.text = df.text.apply(lambda x: re.sub(bracket_regex, "", x))
df.text.loc[df.text.apply(lambda x: x.startswith("Dear"))].iloc[3]

'Dear Senator,\n\nI am writing to express my support for retaining the Electoral College as the method for electing the President of the United States. While critics argue for its abolition, citing concerns about fairness and potential issues, it is crucial to consider the nuanced advantages highlighted by various sources, including the Office of the Federal Register and Judge Richard A. Posner.\n\nThe Electoral College, as explained by the Federal Register, was established as a compromise between direct congressional election and a popular vote, ensuring a balanced representation of federal and state interests (Federal Register, 1). This compromise remains relevant today, preventing the domination of densely populated areas and fostering a more inclusive system.\n\nCritics, such as Bradford Plumer, point out flaws in the Electoral College, emphasizing the potential for elector confusion and the so-called "disaster factor" (Plumer, 10-11). However, Judge Posner provides a more balanced

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections.abc import Iterable


# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
def remove_stopwords(words: str, stopwords: Iterable[str]) -> str:
    word_tokens = word_tokenize(words)
    # converts the words in word_tokens to lower case and then checks whether 
    #they are present in stop_words or not
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    #with no lower case conversion
    filtered_sentence = []
     
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return " ".join(filtered_sentence)


stop_words = set(stopwords.words('english'))
df.text = df.text.apply(lambda x: remove_stopwords(x, stop_words))
df.text.loc[df.text.apply(lambda x: x.startswith("Dear"))].iloc[3]

"Dear Senator , I writing express support retaining Electoral College method electing President United States . While critics argue abolition , citing concerns fairness potential issues , crucial consider nuanced advantages highlighted various sources , including Office Federal Register Judge Richard A. Posner . The Electoral College , explained Federal Register , established compromise direct congressional election popular vote , ensuring balanced representation federal state interests ( Federal Register , 1 ) . This compromise remains relevant today , preventing domination densely populated areas fostering inclusive system . Critics , Bradford Plumer , point flaws Electoral College , emphasizing potential elector confusion so-called `` disaster factor '' ( Plumer , 10-11 ) . However , Judge Posner provides balanced perspective , outlining practical reasons retaining Electoral College , including certainty outcome avoidance runoff elections ( Posner , 18 , 22 ) . The certainty outcome

In [8]:
(df.generated == 1).sum()

115

In [9]:
df.isnull().values.any()

False

In [10]:
df.dtypes

text         object
prompt_id    object
generated    object
dtype: object

In [11]:
df.prompt_id = df.prompt_id.astype(int)
df.generated = df.generated.astype(int)
df.dtypes

text         object
prompt_id     int32
generated     int32
dtype: object

## Classifiers

In [12]:
from sklearn.model_selection import train_test_split


data_train, data_test = train_test_split(df,
                                         train_size=0.75, 
                                         test_size=0.25, 
                                         stratify=df.generated)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer().fit(data_train.text)
x_train = vectorizer.transform(data_train.text)
y_train = data_train.generated.values
x_test = vectorizer.transform(data_test.text)
y_test = data_test.generated.values

The metric we will be using is Macro-F1 average.

- **F1** is a metric used to balance the need for making sure our classifications for a category are both correct (precision) and represent as many of the actual cases of the category as possible (recall).
- **Macro-F1** is the unweighted average of all F1 metrics for each class. We choose Macro F1 instead of a weighted average because we have an unbalanced dataset (Generated essays data are a small fraction of overall essays)


Thus, we want to use a metric which favors both thorough and precise classifiers, and which also assigns equal importance to our smaller classes.

In [14]:
from sklearn.model_selection import cross_val_score


def cross_val_res(model, x, y, scoring=None, cv=10):
    """
    Minor utility method, wraping cross_val_score.
    """
    if scoring is None:
        scoring = "f1_macro"
    res = cross_val_score(model, x, y, cv=cv, scoring=scoring)
    return res.mean(), res.std()

### Dummy Classifier

We will first run a "fake" classifier which only guesses the majority category.

This dummy model thus completely disregards the input features and serves as a useful baseline with which to compare the subsequent classifiers.

In [15]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier


majority_model = DummyClassifier(strategy="most_frequent")
res = cross_val_res(majority_model, x_train, y_train)
print(f"Dummy Classifier mean macro F1-score {res[0]:.4f}, std: {res[1]:.4f}")

Dummy Classifier mean macro F1-score 0.4800, std: 0.0011


### Naive Bayes

Naive Bayes is a very cheap and easy-to-interpret classifier, which checks for the probability that each individual word in the text will belong in any language. We generally want to use the simplest model for the job, and so we start with this reliable model which has proven itself in many fields in the past.

The `sklearn` library gives us access to many variations of Naive Bayes, each specialized in its own field. For this NLP task, we will be using `MultinomialNB`, which was suggested by [this blogpost](https://towardsdatascience.com/naive-bayes-classifiers-for-text-classification-be0d133d35ba).

In [16]:
from sklearn.naive_bayes import MultinomialNB

# naive bayes needs dense arrays to work 
naive_x_train = x_train.toarray()
naive_x_test = x_test.toarray()

naive_model = MultinomialNB()
res = cross_val_res(naive_model, naive_x_train, y_train)
print(f"Naive Bayes mean macro F1-score {res[0]:.4f}, std: {res[1]:.4f}")

Naive Bayes mean macro F1-score 0.5508, std: 0.0776


In [17]:
naive_model = MultinomialNB().fit(x_train, y_train)
naive_res = naive_model.predict(x_test)
print(classification_report(y_test, naive_res, zero_division=0))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97       344
           1       1.00      0.34      0.51        29

    accuracy                           0.95       373
   macro avg       0.97      0.67      0.74       373
weighted avg       0.95      0.95      0.94       373



### Logistic Regression

LogisticRegression despite its name is a linear classifier, meaning that it attempts to linearly separate the data into distinct categories. This interpretation does not apply well to a NLP task, but means that the classifier retains some very useful properties:

- The solution we get is a global optimum, meaning that it's the best we can get with the provided data. This means no hyper-parameter tuning is necessary and we can use the classifier as-is.
- It's a simple and very easy to compute classifier, since it solves a (mathematically simple) linear problem, albeit with some restrictions (technically those restrictions force it to use gradient descent, but the calculations are much easier than say, a neural network)

In [18]:
from sklearn.linear_model import LogisticRegression
import warnings


with warnings.catch_warnings():
    # ignore warnings about deprecated methods in libraries
    warnings.simplefilter("ignore")

    lr = LogisticRegression(max_iter=1000)
    res = cross_val_res(lr, x_train, y_train)
    print(f"Logistic Regression mean macro F1-score {res[0]:.4f}, std: {res[1]:.4f}")

Logistic Regression mean macro F1-score 0.9820, std: 0.0248


In [19]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    lr = LogisticRegression(max_iter=1000).fit(x_train, y_train)
    lr_res = lr.predict(x_test)

print(classification_report(y_test, lr_res, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       344
           1       1.00      1.00      1.00        29

    accuracy                           1.00       373
   macro avg       1.00      1.00      1.00       373
weighted avg       1.00      1.00      1.00       373



### Random Forest

Random Forest is an ensemble algorithm, which means it uses many simpler algorithms which then "vote" on a final decision. It has proven to be a good classifier on complex tasks, it combats overfitting by design (essentially by utilizing random chance in its training phase) and is still fairly easy to interpret.

The drawback is first and foremost computational, since we need to train many smaller classifiers, which may by themselves be computationally expensive (this is somewhat offset by the fact that the classifiers are indepednent and can be computed in parallel). Additionally, Random Forest is a non-parametric method which means that it is generally memory-intensive and may be slow to run on operational data. Finally, we also need to tune hyperparameters.

In [20]:
from sklearn.ensemble import RandomForestClassifier


forest_model = RandomForestClassifier(n_estimators=500,
                                      criterion="entropy")
res = cross_val_res(forest_model, x_train, y_train, cv=5)
print(f"Random Forest mean macro F1: {res[0]:.4f}, std: {res[1]:.4f}")

Random Forest mean macro F1: 0.9869, std: 0.0125


In [21]:
forest_model = RandomForestClassifier(n_estimators=500, 
                                      criterion="entropy").fit(x_train, y_train)
forest_pred = forest_model.predict(x_test)
print(classification_report(y_test, forest_pred, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       344
           1       1.00      1.00      1.00        29

    accuracy                           1.00       373
   macro avg       1.00      1.00      1.00       373
weighted avg       1.00      1.00      1.00       373



### Adaboost

Adaboost is the logical conclusion of Random Forests, where each voter considers a very specific "rule" that needs to be followed. The next voter then considers the most important rule to distinguish between the categories for all the clases that the first could not reliably classify, and so on.

This classifier is generally more compact and competent than a simple Random Forest, but is more computationally expensive during training because we cannot train it in parallel.

In [22]:
from sklearn.ensemble import AdaBoostClassifier


ada_model = AdaBoostClassifier(n_estimators=150)
res = cross_val_res(ada_model, x_train, y_train, cv=3)
print(f"AdaBoost mean macro F1: {res[0]:.4f}, std: {res[1]:.4f}")

AdaBoost mean macro F1: 0.9871, std: 0.0046


In [23]:
from sklearn.ensemble import AdaBoostClassifier


ada_model = AdaBoostClassifier(n_estimators=150).fit(x_train, y_train)
ada_pred = ada_model.predict(x_test)
print(classification_report(y_test, ada_pred, zero_division=0))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       344
           1       1.00      1.00      1.00        29

    accuracy                           1.00       373
   macro avg       1.00      1.00      1.00       373
weighted avg       1.00      1.00      1.00       373

